 Name: Ayush Jain 
 
 Email: ayushjain051001@gmail.com
 
 Phone: 8560036606

# Data Harvesting or Scraping of Walmart Website

You have to harvest brownfield data from a public source. Please write a python harvester in class abstraction and scalable manner (Object Oriented Programming). The output should be tabular, cleanly structured data (.csv or .json), preferably with data for 500-1000 samples. 


## Importing the Libraries
Importing all the required libraries for the program

In [1]:
from selenium.webdriver.support.ui import Select
from dateutil.parser import parse
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
from webdriver_manager.chrome import ChromeDriverManager

## Creating a Scrapper Class which will scrap the data from the pages.

In [15]:
class Scrap:
    def __init__(self):
        # Installing ChromeDriver if not available and will create an object for the same. 
        self.driver = webdriver.Chrome(ChromeDriverManager().install())

    def save_csv(self, date, name, title, sno, desc, rating):
        # All the lists are tranformed to a dictionary for saving the data in a csv file. It helps in understanding the data easily.
        scraped_data = pd.DataFrame({'S.No':sno, 'Title':title, 'Name':name, 'Description':desc, 'Date':date, 'Rating':rating})
        scraped_data.to_csv('result.csv', index=False)
        
    def extract(self, endDate):
        # We declare lists which will store data of each review.
        
        date = []
        name =[]
        title=[]
        desc=[]
        rating=[]
        sno=[]
        num=1
        
        # We try to run in a loop which will go on extracting data until and unless it finds a date less then the specified one or their are no more reviews available
        while True:
            # We convert the page source in a proper manner for easy processing and understanding
            """try:"""
            soup = BeautifulSoup(self.driver.page_source, 'html.parser')

            # We run this loop till we scrap all the reviews from the page. We go through all the components one by one.
            for tags in soup.findAll('div', attrs={'class':'customer-review-body'}):
                # We convert the date to datatime format from the string.
                b = parse(tags.find('span', attrs={'class':'review-date-submissionTime'}).text)
                # It is the break condition as the review being read is older then the specified date
                if b<endDate:
                    break

                # We save the data extracted from all the elements in the array.
                sno.append(num)
                date.append(tags.find('span', attrs={'class':'review-date-submissionTime'}).text)
                # Not available is added in the place where the user hasn't provide any information
                if tags.find('span', attrs={'class':'review-footer-userNickname'}) is not None:
                    name.append(str(tags.find('span', attrs={'class':'review-footer-userNickname'}).text).capitalize())
                else:
                    name.append('Not Available')
                if tags.find('h3', attrs={'class':'review-title'}) is not None:
                    title.append(tags.find('h3', attrs={'class':'review-title'}).text)
                else:
                    title.append('Not Available')
                if tags.find('div', attrs={'class':'review-text'}) is not None:
                    desc.append(tags.find('div', attrs={'class':'review-text'}).text)
                else:
                    desc.append('Not Available')
                if tags.find('span', attrs={'class':'seo-avg-rating'}).text is not None:
                    rating.append(tags.find('span', attrs={'class':'seo-avg-rating'}).text)
                else:
                    rating.append('Not Available')
                num+=1

            # Break condition for the whole scraping process and it will close the browser if the condition holds true.
            if b<endDate:
                self.driver.quit()
                break

            # We navigate to the next page to scrap more reviews using button.
            self.driver.find_element_by_class_name('paginator-btn-next').click()
            """except:
            print('All the reviews has been read')
            break"""
                
        self.save_csv(date, name, title, sno, desc, rating)
                
        
    def find(self, search_url):
        # We open the browser and open the given url.
        self.driver.get(search_url)

        # We navigate to the bottom of the webpage and click on the see all reviews section to navigate to the other page
        self.driver.find_element_by_link_text("See all reviews").click()

        # We sort the items in the order of newest item first by selecting th option from the drop down menu using XPath
        sel = Select(self.driver.find_element_by_xpath('/html/body/div[1]/div/div/div/div[1]/div/div[5]/div/div[2]/div/div[2]/div/div[2]/select'))
        sel.select_by_value('submission-desc')
        self.driver.refresh()

        

        # We need to extract data till 1 December 2021 only. Therefore it will be the end date.
        endDate = datetime(2020, 12, 1)

        self.extract(endDate)

## Using the scraper to scrap reviews from Walmart Website

In [18]:
# We provide the url of the website to scrap data
search_url="https://www.walmart.com/ip/Clorox-Disinfecting-Wipes-225-Count-Value-Pack-Crisp-Lemon-and-Fresh-Scent-3-Pack-75-Count-Each/14898365"

scraper = Scrap()
scraper.find(search_url)



====== WebDriver manager ======
Current google-chrome version is 92.0.4515
Get LATEST driver version for 92.0.4515
Driver [C:\Users\User\.wdm\drivers\chromedriver\win32\92.0.4515.107\chromedriver.exe] found in cache
